In [ ]:
import re
from PyPDF2 import PdfReader

# =========================
# 1️⃣ Load PDF and extract text
# =========================
pdf_path = r"C:\Users\hamza\Desktop\Medical-Chatbot\data\pdfcoffee.com_medical-books-free-pathophysiology-of-disease-an-introduction-to-clinical-medicine-8th-edition-pdf-pdf-free.pdf"  # Replace with your PDF path
reader = PdfReader(pdf_path)
raw_text = ""

for page in reader.pages:
    raw_text += page.extract_text() + "\n"

print("Raw text length:", len(raw_text))

# =========================
# 2️⃣ Clean text
# =========================

def clean_text(text):
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove page numbers like "Page 12" or "12"
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n\d+\n', '\n', text)  # standalone numbers on lines
    
    # Remove headers/footers if known patterns exist (e.g., book title repeated)
    text = re.sub(r'Robbins Basic Pathology.*', '', text)  # example
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

cleaned_text = clean_text(raw_text)

print("Cleaned text length:", len(cleaned_text))

# =========================
# 3️⃣ Save cleaned text to a file
# =========================
with open("cleaned_book.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print("Cleaned text saved to cleaned_book.txt")

Raw text length: 79
Cleaned text length: 0
Cleaned text saved to cleaned_book.txt


In [19]:
import langchain
print(langchain.__version__)

1.1.2


In [4]:
# =========================
# Custom text splitter for RAG
# =========================
def split_text_custom(text, chunk_size=4000, overlap=800):
    """
    Splits text into chunks of chunk_size characters with overlap.
    Approx 1000 tokens ~ 4000 characters.
    """
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap  # move start by chunk_size - overlap
    
    return chunks

# =========================
# Load cleaned text
# =========================
with open("cleaned_book.txt", "r", encoding="utf-8") as f:
    cleaned_text = f.read()

# =========================
# Split text into chunks
# =========================
chunks = split_text_custom(cleaned_text, chunk_size=4000, overlap=800)

print(f"Total chunks created: {len(chunks)}")
print("Sample chunk:\n", chunks[0][:500], "...")


Total chunks created: 0


IndexError: list index out of range

In [26]:
# ------------------------------
# 0️⃣ Imports (keep your existing ones)
# ------------------------------
import psycopg
from psycopg import Cursor
import ollama
from pathlib import Path

# ------------------------------
# 1️⃣ Variables
# ------------------------------
EMBED_MODEL = "embeddinggemma"  # Your Ollama embedding model
db_connection_str = "dbname=medical_rag user=postgres password=1803 host=localhost port=5432"

# If your chunks are already in memory
# chunks = [...]  # list of strings from the previous splitting step

# ------------------------------
# 2️⃣ Helper functions (reuse yours)
# ------------------------------

def calculate_embeddings(corpus: str) -> list[float]:
    response = ollama.embeddings(EMBED_MODEL, corpus)
    return response["embedding"]

def to_pgvector(vec: list[float]) -> str:
    return "[" + ",".join(str(v) for v in vec) + "]"

def save_embedding(corpus: str, embedding: list[float], cursor: Cursor) -> None:
    pg_vec = to_pgvector(embedding)
    cursor.execute(
        """
        INSERT INTO embeddings (corpus, embedding)
        VALUES (%s, %s::vector)
        """,
        (corpus, pg_vec),
    )

def similar_corpus(input_corpus: str, k: int, cursor: Cursor):
    embedding = calculate_embeddings(input_corpus)
    pg_vec = to_pgvector(embedding)

    cursor.execute(
        """
        SELECT id, corpus, embedding <=> %s::vector AS distance
        FROM embeddings
        ORDER BY distance ASC
        LIMIT %s
        """,
        (pg_vec, k),
    )

    return cursor.fetchall()

# ------------------------------
# 3️⃣ Store chunk embeddings in PostgreSQL
# ------------------------------
with psycopg.connect(db_connection_str) as conn:
    conn.autocommit = True

    with conn.cursor() as cur:
        # Drop old table if exists
        cur.execute("DROP TABLE IF EXISTS embeddings")

        # Create extension pgvector
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

        # Create embeddings table
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS embeddings (
                id SERIAL PRIMARY KEY,
                corpus TEXT,
                embedding VECTOR(768)
            );
            """
        )

        # Iterate through your chunks
        for i, chunk in enumerate(chunks):
            emb = calculate_embeddings(chunk)
            save_embedding(chunk, emb, cur)
            if i % 50 == 0:
                print(f"Processed chunk {i+1}/{len(chunks)}")

        conn.commit()

        # Optional: test similarity search
        print("\n--- Test similarity ---")
        test_results = similar_corpus("What causes inflammation?", 3, cur)
        for r in test_results:
            print(r)


Processed chunk 1/1005
Processed chunk 51/1005
Processed chunk 101/1005
Processed chunk 151/1005
Processed chunk 201/1005
Processed chunk 251/1005
Processed chunk 301/1005
Processed chunk 351/1005
Processed chunk 401/1005
Processed chunk 451/1005
Processed chunk 501/1005
Processed chunk 551/1005
Processed chunk 601/1005
Processed chunk 651/1005
Processed chunk 701/1005
Processed chunk 751/1005
Processed chunk 801/1005
Processed chunk 851/1005
Processed chunk 901/1005
Processed chunk 951/1005
Processed chunk 1001/1005

--- Test similarity ---
(852, 'pression on local vessel endothelium to facilitate neutrophil adhesion and migration and are also potent chemoattractants for neutrophils. Neutrophils also amplify their own recruitment by releasing leukotriene LTB 4 upon urate crystal phagocytosis (see Figure 24–2 ). FIGURE 24–2 The mechanisms of the initiation and amplification of the acute inflammatory response in gout involve both cytokines and humoral mediators. The intense inflammatory

In [ ]:
import psycopg
import ollama
from groq import Groq
import os
import base64
from pathlib import Path
import time

# ------------------------------
# Variables
# ------------------------------
EMBED_MODEL = "embeddinggemma"
LLM_MODEL = "llama3"
VISION_MODEL = "llama3.2-vision"
GROQ_MODEL = "llama-3.3-70b-versatile"
GROQ_API_KEY = "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"

db_connection_str = "dbname=rag_chatbot user=postgres password=1803 host=localhost port=5432"
TOP_K = 5

groq_client = Groq(api_key=GROQ_API_KEY)

# ------------------------------
# Helper functions
# ------------------------------

def calculate_embeddings(corpus: str) -> list[float]:
    """Get embeddings from Ollama embedding model."""
    response = ollama.embeddings(EMBED_MODEL, corpus)
    return response["embedding"]

def to_pgvector(vec: list[float]) -> str:
    """Convert list[float] to Postgres vector string."""
    return "[" + ",".join(str(v) for v in vec) + "]"

def retrieve_chunks(query: str, k: int = TOP_K):
    """Retrieve top-k most similar chunks from pgvector."""
    embedding = calculate_embeddings(query)
    pg_vec = to_pgvector(embedding)

    with psycopg.connect(db_connection_str) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                SELECT corpus, embedding <=> %s::vector AS distance
                FROM embeddings
                ORDER BY distance ASC
                LIMIT %s
                """,
                (pg_vec, k)
            )
            results = cur.fetchall()
    
    return [r[0] for r in results]

def encode_image_to_base64(image_path: str) -> str:
    """Convert image file to base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# ------------------------------
# AGENT 1: Vision Agent (Llama 3.2 Vision) - SIMPLIFIED
# ------------------------------
def vision_agent(image_path: str) -> dict:
    """Simple image description for educational project."""
    print("   👁️  Getting image description...")
    
    try:
        if not os.path.exists(image_path):
            return {
                "success": False,
                "error": f"Image file not found: {image_path}",
                "description": None
            }
        
        file_size = os.path.getsize(image_path) / (1024 * 1024)
        print(f"   📂 Image: {os.path.basename(image_path)} ({file_size:.2f} MB)")
        
        print(f"   🔄 Processing...")
        image_base64 = encode_image_to_base64(image_path)
        
        print("   🤖 Analyzing image (30-60 sec)...")
        start_time = time.time()
        
        # SIMPLIFIED PROMPT - Just describe what you see
        response = ollama.chat(
            model=VISION_MODEL,
            messages=[
                {
                    "role": "user",
                    "content": """This is for an educational AI project. Simply describe what you see in this brain scan image:

- What type of medical image is this?
- What structures or features are visible?
- Describe any notable areas, shapes, or patterns you observe
- Note the colors, contrasts, and any distinct regions

Just give a simple, factual description of what's visible in the image.""",
                    "images": [image_base64]
                }
            ]
        )
        
        elapsed = time.time() - start_time
        print(f"   ✓ Analysis complete ({elapsed:.1f}s)")
        
        description = response['message']['content']
        
        return {
            "success": True,
            "description": description,
            "image_path": image_path
        }
    
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e),
            "description": None
        }

# ------------------------------
# AGENT 2: Tumor Classification Agent (LLaMA 3 + RAG)
# ------------------------------
def tumor_classification_agent(vision_description: str) -> dict:
    """Classify tumor type based on description - EDUCATIONAL PROJECT."""
    print("   🔬 Searching medical database...")
    
    # Search for relevant medical information
    search_query = f"brain tumor types classification characteristics symptoms {vision_description[:200]}"
    chunks = retrieve_chunks(search_query, TOP_K)
    
    print(f"   ✓ Found {len(chunks)} relevant sources")
    
    context = "\n\n".join(chunks) if chunks else "Limited textbook information available."
    
    print("   🧠 Analyzing with AI...")
    
    messages = [
        {
            "role": "system", 
            "content": "You are helping with an educational AI project about brain tumor classification. Provide informative, educational content."
        },
        {
            "role": "user",
            "content": f"""This is for an educational AI/ML project demonstrating multi-agent systems.

IMAGE DESCRIPTION:
{vision_description}

MEDICAL REFERENCE MATERIAL:
{context}

Based on the image description, provide an EDUCATIONAL analysis covering:

1. **Possible Tumor Type** (educational guess based on description)
   - What this might resemble
   - Common characteristics

2. **Key Features Observed**
   - What the description suggests
   - Typical patterns for this type

3. **General Information** 
   - How these tumors typically present
   - Common locations and characteristics

4. **Educational Context**
   - Why certain features are significant
   - What medical professionals look for

Remember: This is a STUDENT PROJECT for learning about AI systems, not actual medical diagnosis.
"""
        }
    ]
    
    response = ollama.chat(LLM_MODEL, messages=messages)
    classification = response['message']['content']
    
    print("   ✓ Analysis complete!")
    
    return {
        "classification": classification,
        "textbook_chunks_used": len(chunks)
    }

# ------------------------------
# AGENT 3: Educational Information Agent (Groq)
# ------------------------------
def recommendation_agent(vision_description: str, classification: str) -> str:
    """Provide educational medical information using Groq."""
    print("   📚 Generating educational content...")
    
    try:
        completion = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You are an educational medical AI helping students learn about brain tumors and medical AI systems. Provide informative, educational content."
                },
                {
                    "role": "user",
                    "content": f"""This is for a STUDENT PROJECT demonstrating multi-agent AI systems. NOT for actual medical use.

IMAGE DESCRIPTION:
{vision_description}

AI CLASSIFICATION:
{classification}

Provide EDUCATIONAL INFORMATION about this type of condition:

═══════════════════════════════════════════════════════════════
📚 EDUCATIONAL OVERVIEW
═══════════════════════════════════════════════════════════════
General information about this type of brain tumor for learning purposes.

═══════════════════════════════════════════════════════════════
🔬 TYPICAL CHARACTERISTICS
═══════════════════════════════════════════════════════════════
What features are typically seen with this condition.

═══════════════════════════════════════════════════════════════
🏥 GENERAL TREATMENT APPROACHES
═══════════════════════════════════════════════════════════════
Common treatment methods used for these conditions (educational overview):
- Surgical approaches
- Radiation options
- Medication therapies
- Other interventions

═══════════════════════════════════════════════════════════════
📊 MEDICAL PROCESS
═══════════════════════════════════════════════════════════════
How doctors typically approach diagnosis and treatment (for learning):
- Diagnostic steps
- Specialist involvement
- Treatment planning
- Follow-up care

═══════════════════════════════════════════════════════════════
💡 LEARNING POINTS
═══════════════════════════════════════════════════════════════
Key takeaways for understanding this condition and how AI can assist in medical education.

CLEARLY STATE: This is educational content for a student AI project, not medical advice.
"""
                }
            ],
            temperature=0.7,
            max_tokens=2500
        )
        
        print("   ✓ Educational content generated!")
        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return f"Unable to generate educational content: {str(e)}"

# ------------------------------
# Multi-Agent Brain Tumor Analysis Pipeline
# ------------------------------
def analyze_brain_tumor(image_path: str) -> dict:
    """Educational brain tumor analysis demo."""
    
    print("\n" + "╔" + "═"*68 + "╗")
    print("║" + " 🎓 EDUCATIONAL AI PROJECT - Brain Tumor Analysis Demo ".center(68) + "║")
    print("╚" + "═"*68 + "╝")
    
    # AGENT 1: Vision Analysis
    print("\n" + "─"*70)
    print("🤖 AGENT 1: Vision Description")
    print("─"*70)
    
    vision_result = vision_agent(image_path)
    
    if not vision_result["success"]:
        return {
            "success": False,
            "error": vision_result['error']
        }
    
    # AGENT 2: Classification
    print("\n" + "─"*70)
    print("🤖 AGENT 2: Medical Knowledge Analysis")
    print("─"*70)
    
    classification_result = tumor_classification_agent(vision_result["description"])
    
    # AGENT 3: Educational Info
    print("\n" + "─"*70)
    print("🤖 AGENT 3: Educational Information")
    print("─"*70)
    
    recommendations = recommendation_agent(
        vision_result["description"],
        classification_result["classification"]
    )
    
    return {
        "success": True,
        "vision_analysis": vision_result["description"],
        "tumor_classification": classification_result["classification"],
        "recommendations": recommendations,
        "textbook_sources": classification_result["textbook_chunks_used"],
        "image_path": image_path
    }

# ------------------------------
# Text-Based Query Functions
# ------------------------------
def route_query(query: str) -> str:
    """Route text queries to categories."""
    messages = [
        {"role": "system", "content": "You are a medical query classifier."},
        {
            "role": "user",
            "content": f"""Classify into ONE category: textbook, medication, emergency, lifestyle, or general.
Query: {query}
Respond with ONLY the category name."""
        }
    ]
    
    response = ollama.chat(LLM_MODEL, messages=messages)
    category = response['message']['content'].strip().lower()
    
    valid = ["textbook", "medication", "emergency", "lifestyle", "general"]
    return category if category in valid else "general"

def textbook_agent(query: str) -> str:
    """Answer using textbook RAG."""
    print("   📚 Searching medical textbook...")
    chunks = retrieve_chunks(query, TOP_K)
    
    if not chunks:
        return "No relevant information found in textbook."
    
    context = "\n\n".join(chunks)
    
    messages = [
        {"role": "system", "content": "You are a medical textbook expert."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer based on the context:"}
    ]
    
    response = ollama.chat(LLM_MODEL, messages=messages)
    return response['message']['content']

def enrichment_agent(query: str, textbook_answer: str, category: str) -> str:
    """Enrich answer with Groq."""
    print("   🌟 Enriching with additional info...")
    
    try:
        completion = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {"role": "system", "content": "You are a medical expert educator."},
                {"role": "user", "content": f"Question: {query}\n\nTextbook: {textbook_answer}\n\nEnrich with practical advice and context:"}
            ],
            temperature=0.7,
            max_tokens=1024
        )
        return completion.choices[0].message.content
    except Exception as e:
        return textbook_answer + f"\n\n(Additional enrichment unavailable)"

def ask_multiagent_text(query: str) -> dict:
    """Handle text queries."""
    category = route_query(query)
    textbook_answer = textbook_agent(query)
    enriched_answer = enrichment_agent(query, textbook_answer, category)
    
    return {
        "category": category,
        "textbook_answer": textbook_answer,
        "final_answer": enriched_answer
    }

# ------------------------------
# Main Interactive Loop
# ------------------------------
def main():
    print("\n" + "╔" + "═"*68 + "╗")
    print("║" + " 🎓 EDUCATIONAL MEDICAL AI PROJECT ".center(68) + "║")
    print("╠" + "═"*68 + "╣")
    print("║  Multi-Agent System Demo:".ljust(69) + "║")
    print("║    🧠 Brain Scan Analysis (Vision + RAG + AI)".ljust(69) + "║")
    print("║    💬 Medical Q&A (RAG-based)".ljust(69) + "║")
    print("╠" + "═"*68 + "╣")
    print("║  Commands:".ljust(69) + "║")
    print("║    'analyze <image_path>' - Analyze brain scan".ljust(69) + "║")
    print("║    Just type your question - Medical Q&A".ljust(69) + "║")
    print("║    'exit', 'quit', 'q' - Exit program".ljust(69) + "║")
    print("╠" + "═"*68 + "╣")
    print("║  ⚠️  EDUCATIONAL USE ONLY - Not for medical diagnosis".ljust(69) + "║")
    print("╚" + "═"*68 + "╝\n")
    
    while True:
        user_input = input("💬 You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'q']:
            print("\n👋 Thanks for testing this educational AI project!\n")
            break
        
        if not user_input:
            continue
        
        # Check for brain scan analysis
        if user_input.lower().startswith(('analyze ', 'image ', 'scan ')):
            parts = user_input.split(maxsplit=1)
            if len(parts) < 2:
                print("❌ Please provide image path: analyze <path_to_image>\n")
                continue
            
            image_path = parts[1].strip().strip('"').strip("'")
            
            print(f"\n🔍 Starting analysis of: {image_path}")
            
            try:
                result = analyze_brain_tumor(image_path)
                
                if not result["success"]:
                    print(f"\n❌ Analysis failed: {result['error']}\n")
                    continue
                
                # Display results
                print("\n\n" + "╔" + "═"*68 + "╗")
                print("║" + " 📊 EDUCATIONAL ANALYSIS RESULTS ".center(68) + "║")
                print("╚" + "═"*68 + "╝\n")
                
                print("╔" + "═"*68 + "╗")
                print("║ 👁️  IMAGE DESCRIPTION (Agent 1)".ljust(69) + "║")
                print("╚" + "═"*68 + "╝")
                print(result["vision_analysis"])
                
                print("\n\n" + "╔" + "═"*68 + "╗")
                print("║ 🔬 MEDICAL ANALYSIS (Agent 2)".ljust(69) + "║")
                print(f"║ 📚 Used {result['textbook_sources']} reference sources".ljust(69) + "║")
                print("╚" + "═"*68 + "╝")
                print(result["tumor_classification"])
                
                print("\n\n" + "╔" + "═"*68 + "╗")
                print("║ 📚 EDUCATIONAL INFORMATION (Agent 3)".ljust(69) + "║")
                print("╚" + "═"*68 + "╝")
                print(result["recommendations"])
                
                print("\n\n" + "╔" + "═"*68 + "╗")
                print("║ 🎓 STUDENT PROJECT DISCLAIMER ".center(68) + "║")
                print("╠" + "═"*68 + "╣")
                print("║  This is an EDUCATIONAL AI project demonstration".ljust(69) + "║")
                print("║  NOT for actual medical diagnosis or advice".ljust(69) + "║")
                print("║  For real medical concerns, consult healthcare professionals".ljust(69) + "║")
                print("╚" + "═"*68 + "╝\n")
                
            except Exception as e:
                print(f"\n❌ Error: {e}\n")
                import traceback
                traceback.print_exc()
        
        # Text-based query
        else:
            try:
                print("\n🔍 Processing your question...")
                result = ask_multiagent_text(user_input)
                
                print("\n" + "═"*70)
                print(f"📋 Category: {result['category'].upper()}")
                print("═"*70)
                
                print("\n📚 Textbook Answer:")
                print("-"*70)
                print(result['textbook_answer'])
                
                print("\n✨ Enriched Answer:")
                print("-"*70)
                print(result['final_answer'])
                print("\n" + "═"*70 + "\n")
                
            except Exception as e:
                print(f"\n❌ Error: {e}\n")


if __name__ == "__main__":
    main()


╔════════════════════════════════════════════════════════════════════╗
║                  🎓 EDUCATIONAL MEDICAL AI PROJECT                  ║
╠════════════════════════════════════════════════════════════════════╣
║  Multi-Agent System Demo:                                          ║
║    🧠 Brain Scan Analysis (Vision + RAG + AI)                       ║
║    💬 Medical Q&A (RAG-based)                                       ║
╠════════════════════════════════════════════════════════════════════╣
║  Commands:                                                         ║
║    'analyze <image_path>' - Analyze brain scan                     ║
║    Just type your question - Medical Q&A                           ║
║    'exit', 'quit', 'q' - Exit program                              ║
╠════════════════════════════════════════════════════════════════════╣
║  ⚠️  EDUCATIONAL USE ONLY - Not for medical diagnosis              ║
╚════════════════════════════════════════════════════════════════════╝


🔍 S

In [ ]:
import psycopg
import ollama
from groq import Groq
import os
import base64
from pathlib import Path
import time
from datetime import datetime

# ------------------------------
# Variables
# ------------------------------
EMBED_MODEL = "embeddinggemma"
LLM_MODEL = "llama3"
VISION_MODEL = "llama3.2-vision"
GROQ_MODEL = "llama-3.3-70b-versatile"
GROQ_API_KEY = "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"

db_connection_str = "dbname=rag_chatbot user=postgres password=1803 host=localhost port=5432"
TOP_K = 5

groq_client = Groq(api_key=GROQ_API_KEY)

# Create results directory
RESULTS_DIR = "analysis_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# ------------------------------
# Helper functions
# ------------------------------

def calculate_embeddings(corpus: str) -> list[float]:
    """Get embeddings from Ollama embedding model."""
    response = ollama.embeddings(EMBED_MODEL, corpus)
    return response["embedding"]

def to_pgvector(vec: list[float]) -> str:
    """Convert list[float] to Postgres vector string."""
    return "[" + ",".join(str(v) for v in vec) + "]"

def retrieve_chunks(query: str, k: int = TOP_K):
    """Retrieve top-k most similar chunks from pgvector."""
    embedding = calculate_embeddings(query)
    pg_vec = to_pgvector(embedding)

    with psycopg.connect(db_connection_str) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                SELECT corpus, embedding <=> %s::vector AS distance
                FROM embeddings
                ORDER BY distance ASC
                LIMIT %s
                """,
                (pg_vec, k)
            )
            results = cur.fetchall()
    
    return [r[0] for r in results]

def encode_image_to_base64(image_path: str) -> str:
    """Convert image file to base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def save_results_to_file(result: dict, filename: str):
    """Save analysis results to a text file."""
    filepath = os.path.join(RESULTS_DIR, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("BRAIN SCAN ANALYSIS - EDUCATIONAL AI PROJECT\n")
        f.write("="*80 + "\n")
        f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Image: {result['image_path']}\n")
        f.write("="*80 + "\n\n")
        
        f.write("="*80 + "\n")
        f.write("AGENT 1: WHAT THE AI SEES IN THE IMAGE\n")
        f.write("="*80 + "\n")
        f.write(result['vision_analysis'])
        f.write("\n\n")
        
        f.write("="*80 + "\n")
        f.write("AGENT 2: INFORMATION FROM MEDICAL TEXTBOOK\n")
        f.write(f"(Retrieved from {result['textbook_sources']} sources)\n")
        f.write("="*80 + "\n")
        f.write(result['tumor_classification'])
        f.write("\n\n")
        
        f.write("="*80 + "\n")
        f.write("AGENT 3: ADDITIONAL EDUCATIONAL INFO\n")
        f.write("="*80 + "\n")
        f.write(result['recommendations'])
        f.write("\n\n")
        
        f.write("="*80 + "\n")
        f.write("NOTE: This is a student AI project demo - Not medical advice\n")
        f.write("="*80 + "\n")
    
    return filepath

# ------------------------------
# AGENT 1: Vision Agent - SUPER SIMPLE
# ------------------------------
def vision_agent(image_path: str) -> dict:
    """Just describe what you see in the image - no medical analysis."""
    print("   👁️  Looking at the image...")
    
    try:
        if not os.path.exists(image_path):
            return {
                "success": False,
                "error": f"Image not found: {image_path}",
                "description": None
            }
        
        file_size = os.path.getsize(image_path) / (1024 * 1024)
        print(f"   📂 File: {os.path.basename(image_path)} ({file_size:.2f} MB)")
        
        print(f"   🔄 Reading image...")
        image_base64 = encode_image_to_base64(image_path)
        
        print("   🤖 Describing what I see (30-60 sec)...")
        start_time = time.time()
        
        # SUPER SIMPLE PROMPT - Just describe what you see!
        response = ollama.chat(
            model=VISION_MODEL,
            messages=[
                {
                    "role": "user",
                    "content": """Look at this brain scan image and simply describe what you see. Think of it like describing a picture to someone.

Tell me:
- Is this a black and white or color image?
- What shapes do you see? (circles, ovals, irregular shapes)
- Where are these shapes located? (center, left side, right side, top, bottom)
- Are there any bright spots or dark spots?
- Are there any small circles or dots visible inside?
- What's the background like?
- Any other notable visual features?

Just describe the visual elements you can see - like you're describing any regular image, not a medical scan. Simple, straightforward description.""",
                    "images": [image_base64]
                }
            ]
        )
        
        elapsed = time.time() - start_time
        print(f"   ✓ Done! ({elapsed:.1f} seconds)")
        
        description = response['message']['content']
        
        return {
            "success": True,
            "description": description,
            "image_path": image_path
        }
    
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        return {
            "success": False,
            "error": str(e),
            "description": None
        }

# ------------------------------
# AGENT 2: Knowledge Agent
# ------------------------------
def tumor_classification_agent(vision_description: str) -> dict:
    """Search medical knowledge based on what was seen."""
    print("   🔬 Searching medical textbook...")
    
    search_query = f"brain tumor appearance characteristics {vision_description[:250]}"
    chunks = retrieve_chunks(search_query, TOP_K)
    
    print(f"   ✓ Found {len(chunks)} relevant sections")
    
    context = "\n\n".join(chunks) if chunks else "No specific information found."
    
    print("   🧠 Analyzing...")
    
    messages = [
        {
            "role": "system", 
            "content": "You help students learn about medical conditions. Provide educational information."
        },
        {
            "role": "user",
            "content": f"""Student AI project - educational purposes only.

VISUAL DESCRIPTION FROM IMAGE:
{vision_description}

TEXTBOOK INFORMATION:
{context}

Based on what was observed in the image and the textbook info, provide:

1. **What This Might Look Like**
   Based on the visual description, what types of conditions have similar appearances?

2. **Key Visual Features**
   What features in the description are notable?

3. **General Educational Info**
   What does the textbook say about these types of findings?

Keep it educational and informative. This is for learning about AI + medical knowledge, not diagnosis.
"""
        }
    ]
    
    response = ollama.chat(LLM_MODEL, messages=messages)
    classification = response['message']['content']
    
    print("   ✓ Complete!")
    
    return {
        "classification": classification,
        "textbook_chunks_used": len(chunks)
    }

# ------------------------------
# AGENT 3: Info Agent
# ------------------------------
def recommendation_agent(vision_description: str, classification: str) -> str:
    """Provide additional educational context."""
    print("   📚 Getting more info...")
    
    try:
        completion = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You help students learn about medical AI systems. Provide educational content."
                },
                {
                    "role": "user",
                    "content": f"""This is a STUDENT PROJECT demonstrating AI systems. Educational only.

WHAT WAS SEEN:
{vision_description}

TEXTBOOK INFO:
{classification}

Provide brief educational information about:

1. **What This Demonstrates**
   How this shows AI can process medical images

2. **General Medical Context**
   Basic info about these types of scans and findings

3. **How Doctors Actually Work**
   What real doctors do with these scans (for education)

4. **Learning Takeaway**
   What students can learn from this demo

Keep it short and educational. Remind that this is a demo project, not real medical analysis.
"""
                }
            ],
            temperature=0.7,
            max_tokens=1500
        )
        
        print("   ✓ Done!")
        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return f"Could not generate additional info: {str(e)}"

# ------------------------------
# Analysis Pipeline
# ------------------------------
def analyze_brain_tumor(image_path: str) -> dict:
    """Run the 3-agent analysis pipeline."""
    
    print("\n" + "╔" + "═"*68 + "╗")
    print("║" + " 🧠 3-AGENT IMAGE ANALYSIS DEMO ".center(68) + "║")
    print("╚" + "═"*68 + "╝")
    
    # AGENT 1: Vision
    print("\n" + "─"*70)
    print("🤖 AGENT 1: Describe What You See")
    print("─"*70)
    
    vision_result = vision_agent(image_path)
    
    if not vision_result["success"]:
        return {
            "success": False,
            "error": vision_result['error']
        }
    
    # AGENT 2: Knowledge Search
    print("\n" + "─"*70)
    print("🤖 AGENT 2: Search Medical Textbook")
    print("─"*70)
    
    classification_result = tumor_classification_agent(vision_result["description"])
    
    # AGENT 3: Additional Info
    print("\n" + "─"*70)
    print("🤖 AGENT 3: Additional Context")
    print("─"*70)
    
    recommendations = recommendation_agent(
        vision_result["description"],
        classification_result["classification"]
    )
    
    return {
        "success": True,
        "vision_analysis": vision_result["description"],
        "tumor_classification": classification_result["classification"],
        "recommendations": recommendations,
        "textbook_sources": classification_result["textbook_chunks_used"],
        "image_path": image_path
    }

# ------------------------------
# Text Query Functions
# ------------------------------
def route_query(query: str) -> str:
    messages = [
        {"role": "system", "content": "Classify medical queries."},
        {"role": "user", "content": f"Category (textbook/medication/emergency/lifestyle/general): {query}"}
    ]
    
    response = ollama.chat(LLM_MODEL, messages=messages)
    category = response['message']['content'].strip().lower()
    
    valid = ["textbook", "medication", "emergency", "lifestyle", "general"]
    return category if category in valid else "general"

def textbook_agent(query: str) -> str:
    print("   📚 Searching...")
    chunks = retrieve_chunks(query, TOP_K)
    
    if not chunks:
        return "No info found."
    
    context = "\n\n".join(chunks)
    messages = [
        {"role": "system", "content": "Medical educator."},
        {"role": "user", "content": f"Context:\n{context}\n\nQ: {query}\nA:"}
    ]
    
    response = ollama.chat(LLM_MODEL, messages=messages)
    return response['message']['content']

def enrichment_agent(query: str, textbook_answer: str) -> str:
    print("   🌟 Enriching...")
    
    try:
        completion = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {"role": "system", "content": "Medical expert."},
                {"role": "user", "content": f"Q: {query}\nTextbook: {textbook_answer}\nAdd practical info:"}
            ],
            temperature=0.7,
            max_tokens=800
        )
        return completion.choices[0].message.content
    except:
        return textbook_answer

def ask_multiagent_text(query: str) -> dict:
    category = route_query(query)
    textbook_answer = textbook_agent(query)
    enriched_answer = enrichment_agent(query, textbook_answer)
    
    return {
        "category": category,
        "textbook_answer": textbook_answer,
        "final_answer": enriched_answer
    }

# ------------------------------
# Main
# ------------------------------
def main():
    print("\n" + "╔" + "═"*68 + "╗")
    print("║" + " 🎓 MULTI-AGENT MEDICAL AI DEMO ".center(68) + "║")
    print("╠" + "═"*68 + "╣")
    print("║  'analyze <path>' - Run 3-agent image analysis".ljust(69) + "║")
    print("║  'exit' - Quit".ljust(69) + "║")
    print("╚" + "═"*68 + "╝\n")
    
    while True:
        user_input = input("💬 You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'q']:
            print("\n👋 Thanks for testing!\n")
            break
        
        if not user_input:
            continue
        
        # Image analysis
        if user_input.lower().startswith(('analyze ', 'image ', 'scan ')):
            parts = user_input.split(maxsplit=1)
            if len(parts) < 2:
                print("❌ Usage: analyze <path>\n")
                continue
            
            image_path = parts[1].strip().strip('"').strip("'")
            
            print(f"\n🔍 Starting analysis: {image_path}")
            
            try:
                result = analyze_brain_tumor(image_path)
                
                if not result["success"]:
                    print(f"\n❌ Failed: {result['error']}\n")
                    continue
                
                # Save to file
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"analysis_{timestamp}.txt"
                filepath = save_results_to_file(result, filename)
                
                # Print results
                print("\n\n" + "╔" + "═"*68 + "╗")
                print("║" + " ✅ ANALYSIS COMPLETE ".center(68) + "║")
                print("╚" + "═"*68 + "╝\n")
                
                print("👁️  WHAT THE AI SEES:")
                print("-"*70)
                print(result["vision_analysis"])
                print()
                
                print("📚 TEXTBOOK INFO:")
                print("-"*70)
                print(result["tumor_classification"])
                print()
                
                print("💡 ADDITIONAL CONTEXT:")
                print("-"*70)
                print(result["recommendations"])
                print()
                
                print("="*70)
                print(f"📄 Full results saved: {filepath}")
                print("="*70 + "\n")
                
            except Exception as e:
                print(f"\n❌ Error: {e}\n")
                import traceback
                traceback.print_exc()
        
        # Text query
        else:
            try:
                result = ask_multiagent_text(user_input)
                
                print(f"\n📋 {result['category'].upper()}")
                print("="*70)
                print("\n📚 Textbook:")
                print(result['textbook_answer'])
                print("\n✨ Enhanced:")
                print(result['final_answer'])
                print("="*70 + "\n")
                
            except Exception as e:
                print(f"\n❌ Error: {e}\n")


if __name__ == "__main__":
    main()


╔════════════════════════════════════════════════════════════════════╗
║                   🎓 MULTI-AGENT MEDICAL AI DEMO                    ║
╠════════════════════════════════════════════════════════════════════╣
║  'analyze <path>' - Run 3-agent image analysis                     ║
║  'exit' - Quit                                                     ║
╚════════════════════════════════════════════════════════════════════╝


🔍 Starting analysis: C:\Users\hamza\Desktop\Medical-Chatbot\data\Y10.jpg

╔════════════════════════════════════════════════════════════════════╗
║                   🧠 3-AGENT IMAGE ANALYSIS DEMO                    ║
╚════════════════════════════════════════════════════════════════════╝

──────────────────────────────────────────────────────────────────────
🤖 AGENT 1: Describe What You See
──────────────────────────────────────────────────────────────────────
   👁️  Looking at the image...
   📂 File: Y10.jpg (0.01 MB)
   🔄 Reading image...
   🤖 Describing what I see